In [2]:
import pandas as pd
import os
import torch
from transformers import Trainer, BertTokenizer
import numpy as np

In [3]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/GrammatiktakDatasets/checkedDatasets/")
data = pd.read_csv("CommaDevelopmentset.csv", sep=";")

In [4]:
X = list(data["comment_text"].values)
Y = list(data["label"].values)

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [5]:
def load_model():
    os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/FineTuneModels/Models/")
    models = ["commaModel2", "commaModel4", "commaModel5", "commaModel6"]
    # punctuation model, padding and scope should match model and dataset
    # should have padding so that every word except the last is checked
    # else the logic in finding "find_comma_mistakes" needs to be changed
    punc_models = []
    for model in models:
        loaded_model = torch.load(model, map_location=torch.device('cpu'))
        punc_models.append(loaded_model)
    scope = 6
    padding = int(scope/2-1)

    punc_trainers = []
    for model in punc_models:
        model.eval()
        model.to(device)
        punc_trainers.append(Trainer(model))
    return punc_trainers, scope, padding, models

device = "mps"
torch.device(device)

device(type='mps')

In [6]:
from tqdm import tqdm

class PunctuationCorrector():
    def __init__(self) -> None:
        self.models, self.scope, self.padding, self.model_names = load_model()
        self.tokenizer = BertTokenizer.from_pretrained("Maltehb/danish-bert-botxo")
    
    def add_padding(self, words):
        return ["<PAD>"]*self.padding + words + ["<PAD>"]*self.padding

    # prepares dataset and get predictions
    def get_predictions(self, test_data) -> list:
        final_predictions = []
        print("Tokenizing")
        tokenized_data = self.tokenizer(test_data, padding=True, truncation=True, max_length=15)
        final_dataset = Dataset(tokenized_data)
        for i in tqdm(range(len(self.models))):
            model = self.models[i]
            print("Predicting")
            raw_predictions, _, _ = model.predict(final_dataset)
            final_prediction = np.argmax(raw_predictions, axis=1)
            final_predictions.append(final_prediction)
        return final_predictions
    
    def get_accuracy_for_each_model(self, X, y):
        predictions_lsts = self.get_predictions(X)
        accuracy = [np.mean(predictions == y) for predictions in predictions_lsts]
        print(*[f"{self.model_names[i]} accuracy: {round(accuracy[i], 4)*100}%" for i in range(len(accuracy))], sep="\n")

puncCorrector = PunctuationCorrector()

In [7]:
puncCorrector.get_accuracy_for_each_model(X, Y)

Tokenizing


  0%|          | 0/4 [00:00<?, ?it/s]

Predicting


  0%|          | 0/170 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:20<01:00, 20.21s/it]

...

 75%|███████▌  | 3/4 [01:03<00:21, 21.53s/it]

Predicting


  0%|          | 0/170 [00:00<?, ?it/s]

100%|██████████| 4/4 [01:23<00:00, 20.80s/it]

commaModel2 accuracy: 0.8294%
commaModel4 accuracy: 0.8257%
commaModel5 accuracy: 0.9669%
commaModel6 accuracy: 0.9743%
